In [1]:
import torch
import torch.nn.functional as F
import numpy as np

In [2]:
input = np.array([[2.0,1.0,0.1]])

In [3]:
tempreture = 1
logits = torch.tensor(input /tempreture)
softmax_score = F.softmax(logits, dim=1)
print(f"temperature = {tempreture} {softmax_score.numpy()}")

temperature = 1 [[0.65900114 0.24243297 0.09856589]]


In [4]:
tempreture =0.1
logits = torch.tensor(input /tempreture)
softmax_score = F.softmax(logits, dim=1)
print(f"temperature = {tempreture} {softmax_score.numpy()}")

temperature = 0.1 [[9.99954597e-01 4.53978684e-05 5.60254205e-09]]


In [5]:
tempreture = 20
logits = torch.tensor(input /tempreture)
softmax_score = F.softmax(logits, dim=1)
print(f"temperature = {tempreture} {softmax_score.numpy()}")

temperature = 20 [[0.34957672 0.33252767 0.31789561]]


In [6]:
# 调用大模型的集中方法
# 1. modelscope
from modelscope import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [7]:
model_path = '../model/Qwen2.5-0.5B-Instruct/'
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)
gen_config = GenerationConfig.from_pretrained(model_path)

The module name  (originally ) is not a valid Python identifier. Please rename the original module to avoid import issues.


In [8]:
tokenizer("你好，are you OK?")

{'input_ids': [108386, 3837, 546, 498, 10402, 30], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [9]:
tokenizer.decode(108386)

'你好'

In [10]:
gen_config

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "repetition_penalty": 1.1,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}

In [11]:
def run_prompt(prompt, temperature =0.1, top_k=20, top_p=0.8, max_new_tokens=2048):
    gen_config.temperature = temperature
    gen_config.top_k =top_k
    gen_config.top_p = top_p

    messages = [
        {"role":"system","content":""},
        {"role":"user","content":prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    print(text)
    print("----------------")
    model_input = tokenizer([text], return_tensors="pt")
    generated_ids = model.generate(model_input.input_ids, max_new_tokens = max_new_tokens,generation_config=gen_config)
    print("model_inputids:",model_input.input_ids)
    print("generated_ids:", generated_ids)
    response_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_input.input_ids, generated_ids)]
    print("response_ids:",response_ids)
    response = tokenizer.batch_decode(response_ids, skip_special_tokens=True)[0]
    return response

prompt = "Hello"

run_prompt(prompt)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|im_start|>system
<|im_end|>
<|im_start|>user
Hello<|im_end|>
<|im_start|>assistant

----------------


/opt/anaconda3/envs/transformer/lib/python3.13/site-packages/transformers/generation/utils.py:2532: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on mps. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('mps') before running `.generate()`.
  warnings.warn(


RuntimeError: Placeholder storage has not been allocated on MPS device!

In [12]:
tokenizer.decode(10)

'+'

In [13]:
in_ids = torch.tensor([[151644,   8948,    198, 151645,    198, 151644,    872,    198, 108386,
         151645,    198, 151644,  77091,    198]])
gen_ids = torch.tensor([[151644,   8948,    198, 151645,    198, 151644,    872,    198, 108386,
         151645,    198, 151644,  77091,    198, 108386,   6313, 104139, 111728,
         103929, 101037,  11319, 151645]])

In [14]:
print(in_ids.shape)
print(gen_ids.shape)

torch.Size([1, 14])
torch.Size([1, 22])


In [15]:
for i, j in zip(in_ids, gen_ids):
    print("i=",i)
    print(i.shape)
    print(len(i))
    print("----------------------")
    print("j=",j)
    print(j.shape)
    z = j[11:]
    print(z)
   


i= tensor([151644,   8948,    198, 151645,    198, 151644,    872,    198, 108386,
        151645,    198, 151644,  77091,    198])
torch.Size([14])
14
----------------------
j= tensor([151644,   8948,    198, 151645,    198, 151644,    872,    198, 108386,
        151645,    198, 151644,  77091,    198, 108386,   6313, 104139, 111728,
        103929, 101037,  11319, 151645])
torch.Size([22])
tensor([151644,  77091,    198, 108386,   6313, 104139, 111728, 103929, 101037,
         11319, 151645])


In [16]:
# 2通过transformer调用模型
from transformers import AutoModel, AutoTokenizer
model_path = '../model/Qwen1.5-0.5B-Chat/'
tokenizer= AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True)

In [17]:
print(model)

Qwen2Model(
  (embed_tokens): Embedding(151936, 1024)
  (layers): ModuleList(
    (0-23): 24 x Qwen2DecoderLayer(
      (self_attn): Qwen2Attention(
        (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
      )
      (mlp): Qwen2MLP(
        (gate_proj): Linear(in_features=1024, out_features=2816, bias=False)
        (up_proj): Linear(in_features=1024, out_features=2816, bias=False)
        (down_proj): Linear(in_features=2816, out_features=1024, bias=False)
        (act_fn): SiLUActivation()
      )
      (input_layernorm): Qwen2RMSNorm((1024,), eps=1e-06)
      (post_attention_layernorm): Qwen2RMSNorm((1024,), eps=1e-06)
    )
  )
  (norm): Qwen2RMSNorm((1024,), eps=1e-06)
  (rotary_emb): Qwen2RotaryEmbedding()
)


In [18]:
from transformers import Qwen2ForCausalLM, Qwen2Tokenizer
# 1. 加载分词器和生成模型（必须用 Qwen2ForCausalLM）
model_path = '../model/Qwen1.5-0.5B-Chat/'
tokenizer = Qwen2Tokenizer.from_pretrained(model_path)  # 模型名可替换为 1.5B/7B 等
model = Qwen2ForCausalLM.from_pretrained(
    model_path,
    device_map="auto",  # 自动分配 GPU/CPU（有GPU会自动使用）
    torch_dtype="auto"  # 自动选择数据类型（节省显存）
)

`torch_dtype` is deprecated! Use `dtype` instead!
The module name  (originally ) is not a valid Python identifier. Please rename the original module to avoid import issues.


In [19]:
# 2. 编码输入（add_special_tokens=True 确保添加模型所需特殊token）
inputs = tokenizer(
    "Hello, who are you?",
    return_tensors="pt",
    add_special_tokens=True
).to(model.device)  # 确保输入和模型在同一设备（GPU/CPU）

# 3. 生成文本（调用 generate 方法，可配置生成参数）
outputs = model.generate(
    **inputs,
    max_new_tokens=100,  # 最大生成token数
    temperature=0.7,     # 随机性（0-1，越小越确定）
    top_p=0.9,           # 核采样（筛选概率前90%的token）
    do_sample=True,      # 启用采样生成（更自然）
    eos_token_id=tokenizer.eos_token_id  # 结束符ID
)
# 4. 解码输出（skip_special_tokens=True 忽略特殊token）
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Hello, who are you?


In [20]:
def chatglm(prompt, models = model):
    response = models.generate(
                                **prompt,
                                max_new_tokens=100,  # 最大生成token数
                                temperature=0.7,     # 随机性（0-1，越小越确定）
                                top_p=0.9,           # 核采样（筛选概率前90%的token）
                                do_sample=True,      # 启用采样生成（更自然）
                                eos_token_id=tokenizer.eos_token_id,  # 结束符ID
    )
    return response

inputs = tokenizer(
    "what you can do?",
    return_tensors="pt",
    add_special_tokens=True
).to(model.device)  # 确保输入和模型在同一设备（GPU/CPU）

response = tokenizer.decode(chatglm(inputs)[0], skip_special_tokens=True)
print(response)


what you can do?


In [21]:
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="../model/Qwen1.5-0.5B-Chat")
response = pipe(messages)
response[0]['generated_text'][1]['content']

Device set to use mps:0


'I am a large language model created by Alibaba Cloud. I am called Qwen.'

In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(
    "../model//Qwen1.5-0.5B-Chat",
    torch_dtype="auto",
    device_map="auto"
).to(device)
tokenizer = AutoTokenizer.from_pretrained("../model/Qwen1.5-0.5B-Chat")

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


In [24]:
print(response)

A large language model is a type of machine learning algorithm that can generate text or speech based on input data. It is trained on a large corpus of text, such as books, articles, or websites, and then learns to recognize patterns in the language used by humans to write them. This allows the model to generate high-quality text in various languages and styles, which can be useful for a variety of applications, such as chatbots, natural language processing (NLP), and sentiment analysis. However, building and maintaining large language models can be complex and require significant computing resources, so they are often only used for research purposes or as a part of larger projects.


In [25]:
import torch

# 验证 MPS 是否可用
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")  #  fallback to CPU
print(f"Using device: {device}")

Using device: mps


In [27]:
from openai import OpenAI

In [28]:
client = OpenAI(
    api_key = 'xx',
    base_url='http://localhost:11434/v1'
)
reponse = client.chat.completions.create(
    model = 'qwen2.5:3b',
    messages = [
        {"role":"system","content":""},
        {"role":"user","content":"你能做什么"}
    ],
    temperature = 0.7,
    stream = True
)
print (reponse)

APIConnectionError: Connection error.

In [126]:
for chunk in reponse:
    print(chunk.choices[0].delta.content, end='', flush=True)
    print(chunk)